## **Decorators & higher order functions**
---
When I first got confronted with decorators in Python I was quite confused with the concept. Coming from Java I had of course seen a decorator `@override` which is Java to say you have overridden some method specified in an interface, in Java these are called annotations. Of course, you can implement the decorator programming pattern in Java, which Wikipedia tells me not to confuse with a Python decorator, I find that too strict. The decorator pattern adds behaviour to a single object, Python decorators adds behaviour to foremostly functions/methods, but can also add /modify behaviour of an entire class, as we will see in this notebook. 

So, from a practical view both the decorator pattern and Python decorators are at minimal similar. There is a glaring difference too, Python decorators are  higher order functions. Java does not know higher order functions. I should first answer the question what it is a higher-order function. A higher-order function is a function that does at least one of the following:
 1. It takes one or more functions as arguments.
 2. It returns a function as its result.

Before we look at a decorator let's look at function composition, for function composition is an example of a higher-order function $f \circ g$  or $g(f(x))$.
The three standard operations on a sequence type (tuple, list, and array); map, filter, and fold (aka reduce) are higher-order functions. Let's see them in action.


In [1]:
tuple(map(lambda x : x**2, range(1,11)))

(1, 4, 9, 16, 25, 36, 49, 64, 81, 100)

In [2]:
tuple(filter(lambda x : x > 50, tuple(map(lambda x : x**2, range(1,11)))))

(64, 81, 100)

In [3]:
from functools import reduce
from operator import mul

# multiplying the numbers bigger than 50. 64 x 81, 100
reduce(mul,tuple(filter(lambda x : x > 50, tuple(map(lambda x : x**2, range(1,11))))))

518400

#### **Closure**
The decorator functions as a closure, a closure is a nested function that references one or more variables from its enclosing scope. This definition probably makes more sense with an example.

In the code below `display` is the closure. nested within `say`, `display` uses `greeting` which is defined outside of the local scope of `display`. However, greeting is part of `say()` which is also encompassing scope for `display`.

The variable with name greeting stays "alive" even if display does nothing.


In [13]:
def say():
    greeting = 'Hello'

    def display():
        print(greeting)
        

    return display()    

In [14]:
say()

Hello



You can access the closure directly, the interpreter knows of the existence, but trying to, does not lead to a result, as `display` need `say` to function.

In [24]:
display() 

## **decorators**
Formally closures make it is possible to give a sub-procedure one or more private variables that remain in existence between procedure calls. You actually see closures quite often in Python as you use a lot of decorators in Python. Decorators can be used in Python as higher-order functions and the decorated function has access to the decorator, the decorator contains the closure. Let's look at an example:

In [26]:
def decorator(func):
    
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        func(*args, **kwargs) # This the fst time
        # Do something after
        return func(*args, **kwargs) #This is the third time 
    return wrapper_decorator

@decorator
def george(name):
    '''This is George his function'''
    print('George is a wooly rhino!')
    return f"hi {name}!"

george('George')

George is a wooly rhino!
George is a wooly rhino!


'hi George!'

This simple decorator does only one thing, it executes the print statement twice.

There is simple boilerplate code that allows you to write any decorator you want:

In [29]:
import functools

def decorator(func): # 1
    @functools.wraps(func) # 2
    def wrapper_decorator(*args, **kwargs): # 3 
        #  4 do something
        return func(*args, **kwargs) # 5
    return wrapper_decorator # 6

#### Code comment
 1. The name of the function to be used as the decorator, here aptly called decorator.
 2. The decorator at `@functools.wraps` ensures that you can inspect your decorated function and not the decorator.
 3. The convention is to call these functions wrapper_decorator_name, which in our case is decorator.
 4. Room to add to the functionality of the decorated function.
 5. If we want the function to do something else outside of the decorator, we need to return the original function with original arguments.
 6. Return for the decorator.

On point three inspecting `george` would result into useless info without the wraps decorator.


In [30]:
george

<function __main__.decorator.<locals>.wrapper_decorator(*args, **kwargs)>

In [31]:
help(george)

Help on function wrapper_decorator in module __main__:

wrapper_decorator(*args, **kwargs)



In [32]:
from functools import wraps

def decorator(func):
    @wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        func(*args, **kwargs) # This the fst time
        func(*args, **kwargs) # This is the snd time
        # Do something after
        return func(*args, **kwargs) #This is the third time 
    return wrapper_decorator

@decorator
def george(name):
    '''This is George's his function'''
    print('George is a rhino!')
    return f"hi {name}!"

In [33]:
george

<function __main__.george(name)>

In [34]:
help(george)

Help on function george in module __main__:

george(name)
    This is George's his function




The primary use of decorators is to enrich functions with outside behaviour. 

Consider the simple fibonacci function below which returns us the n-th fibonacci number

In [53]:
ok = 10 
waisting_time = 40
are_you_crazy = 1000

In [54]:
def fibonacci(n:int)->int:
    if n == 0: return 0
    if n == 1: return 1
    else: return fibonacci(n-1) + fibonacci(n-2)  
fibonacci(ok)

55

As we can see it works, but unfortunately this is not a very efficient function, which we see if we compute the 40th Fibonacci number.

In [55]:
%time fibonacci(waisting_time)

CPU times: total: 42.5 s
Wall time: 42.5 s


102334155

The inefficiency is in that we don't store intermediate results after all we have already computed the eight fibonacci number (34 in 34 + 55  = 89).
We could apply a bit of tabulation and store the intermediate results.

In [62]:
def fibonacci(n:int)->int:
    '''fibonacci with tabulation'''
    table = [None]*(n+1)
    table[0] = 0
    table[1] = 1
    for index in range(2,n+1):
        table[index] = table[index-1] + table[index-2]
    return table[n]
fibonacci(waisting_time)

102334155

This wil surely stop waisting all that time by recalculating intermediate results.  
There is a simple decorator in the functools library with which we enrich our original function to store intermediate results. With caching I can compute the 1000th fibonacci number in a flash

In [63]:
from functools import cache

@cache
def fibonacci(n:int)->int:
    '''this fibonacci method returns the n-th fibonacci number'''
    if n == 0: return 0
    if n == 1: return 1
    else: return fibonacci(n-1) + fibonacci(n-2)  
fibonacci(are_you_crazy)

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875

#### **Library decorators**
Python has many libraries that offer function decorators. Functools for instance, functools offers a function with which we can overload methods, something Python as a dynamically typed language does not support natively. See https://en.wikipedia.org/wiki/Function_overloading.

The idea behind these overloaded function is that you let the compiler decide which one to use at runtime. To be able to make this distinction the compiler needs two things: 
 1. It needs to know this function is overloaded, the decorator `@singledispatch` takes care of this.
 2. It needs to register the different implementations `@fun.register` takes care of that.

In [65]:
from functools import singledispatch

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
    print(arg)

In [66]:
@fun.register
def _(arg: int, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)


@fun.register
def _(arg: list, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i+1, elem)

In [67]:
fun(1)

1


In [68]:
fun(['Ente', 'Rhino', 'Croc', 'George'])

1 Ente
2 Rhino
3 Croc
4 George


As you see the only difference between `fun()` functions is the type of their argument.
This is the only thing the compiler needs to choose between these functions. Other libraries also function decorators. One library that you will use a lot is Pytest, it knows decorators.

Run the code outside of this notebook, as notebooks usually are tested with doctest.

In [72]:
import pytest

@pytest.mark.parametrize("test_input,expected", [("3+5", 8), ("2+4", 6), ("6*9", 42)])
def test_eval(test_input, expected):
    assert eval(test_input) == expected
       

## **Class decorators**
#### Your not bound to only use decorators on functions or methods but you can also use a decorator on a class.
#### With Python 3.7 introduced the dataclasses module with the @dataclass decorator 
#### I would advocate the use of the dataclasses for three reasons
#### 1 Dataclasses bring consistency in developing classes
#### 2 Dataclasses solve the empty list as default value problem
#### 3 Dataclasses allow for the easy use of Python's special methods  

#### **One consistency in developing classes**  
#### Python is a very flexible language, not only in its Typing but also in the way you use it.
#### Dataclasses give you structure in how to develop classes and force concise typing
#### In Python you can set up a class in different manners

In [12]:
class ExampleOne:
    
    def __init__(self, att_one, att_two, att_three):
        self.att_one   = att_one
        self.att_two   = att_two
        self.att_three = att_three     
        
        def __str__(self):
            return f'This class {type(self).__name__} is written without the dataclasses module'
        

In [13]:
one = ExampleOne(1, 'two', True)
one.att_one

1

In [14]:
print(one)

In [15]:
help(one)

Help on ExampleOne in module __main__ object:

class ExampleOne(builtins.object)
 |  ExampleOne(att_one, att_two, att_three)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, att_one, att_two, att_three)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



In [16]:
class ExampleTwo:
    
    def __init__(self):
        self.att_one   = 1
        self.att_two   = 'two'
        self.att_three = True     
        
        def __str__(self):
            return f'This class {type(self).__name__} is written without the dataclasses module'

In [17]:
two = ExampleTwo()
two.att_one

1

In [18]:
print(two)

In [19]:
class ExampleThree:
    att_one = 1
    att_two  = 'two'
    att_three = True  
    
    def __str__(self):
            return f'This class {type(self).__name__} is written without the dataclasses module'
    

In [20]:
three = ExampleThree()
three.att_one

1

In [28]:
class ExampleFour:
    
    def __init__(self, att_one=1, att_two='two', att_three=True):
        self.att_one   = att_one
        self.att_two   = att_two
        self.att_three = att_three    
        
    def __str__(self):
        return f'This class {type(self).__name__} is written without the dataclasses module'
    


In [29]:
four = ExampleFour()
print(four)

This class ExampleFour is written without the dataclasses module


In [30]:
four.att_one

1

#### you can even create a class using type, this is a form of meta programming see that notebook for details

In [31]:
def __str__(self):
    return f'This class {type(self).__name__} is written without the dataclasses module'

ExampleFive = type('ExampleFive', (), {'att_one':1, 'att_two':'two','att_three':True, '__str__':__str__})
five = ExampleFive()
print(five)

This class ExampleFive is written without the dataclasses module


In [32]:
five.att_three

True

#### These are four quite common ways to see a class created in Python (and one very uncommon :-) )
#### These different ways of creating classes are confusing not only for beginning programmers but for everybody there is no consistency here, and it is very buggy. 
#### Python dataclasses overcome this by providing a scaffolding or framework for building classes, if everybody uses that framework, then all classes will be written in a consistent manner. 
#### The scaffolding is build upon the concept of an object, it has traits (attributes) which represent state and it has methods to change that state. Dataclasses follow this theoretical model which makes sure that programmers from different OO langauages would understand this perfectly well, as providing beginners with solid structure

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ExampleFive:
    #attributes
    att_one:int
    att_two:str
    att_three:bool
    
    #methods
    def __str__(self):
        return f'This class {type(self).__name__} is written with the dataclasses module'
    

In [ ]:
five = ExampleFive(att_one=1, att_two='two', att_three=True)
print(five)

#### The dataclasses module is linked with Typing.
#### It enforces you name a type when declaring a field.
#### A field is an attribute with type. 
#### The compiler checks this

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ExampleSix:
    #attributes
    att_one
    att_two:str
    att_three:bool
    
    #methods
    def __str__(self):
        return f'This class {type(self).__name__} is written with the dataclasses module'

#### Python being Python a dynamic language I can still create a class with whatever type I fancy  

In [ ]:
five = ExampleFive(3.7, 'George', 0)
five

#### I would need to use an external type checker as MyPy to catch type errors. 
#### Using MyPy allows you to program static Python. I am not sure if this the way to go in Python. Despite the obvious advantages static languages have faster and less prone to faults, I believe the flexibility of Python might out way it. Having said that I wouldn't write software for a hart monitor in Python, or critical resource for that matter.

#### You can default values in classes with fields. 
#### You need to ensure that fields with default values come after fields with no values
#### And you need to use the field function to create empty list, set or tuples

In [ ]:
from dataclasses import dataclass, field

@dataclass
class ExampleSix:
    #attributes
    att_one:int
    att_two:str
    att_three:bool
    # default values
    att_four:list[str] = field(default_factory=list)
    att_five:set[int] = field(default_factory=set)
    
    #methods
    def __str__(self):
        return f'This class {type(self).__name__} is written with the dataclasses module'

In [ ]:
six = ExampleSix(1, 'George', True)
six

In [ ]:
print(six)

#### **The empty list as default value problem**
#### You have probably seen this problem. Without completely understanding what was happening. 
#### The empty list as default value problem is a nefarious problem for Python to solve and very difficult to understand if your knowledge of storables in Python is weak.
#### Let me show you the problrem, and that `@dataclass` solves it.

In [ ]:
class Taxi:
    '''Class to illustrate the empty default list problem'''
    
    def __init__(self, passengers=[]): # de taxi is initially empty
        self.passengers=passengers
        
    def pick_up(self,name:str):
        self.passengers.append(name)
    
    def drop_off(self,name:str):
        self.passengers.remove(name)  

#### Let's make a taxi

In [ ]:
taxi_one = Taxi(passengers=['Ente', 'Croc'])
taxi_one.passengers

In [ ]:
taxi_one.drop_off('Ente')
taxi_one.passengers

In [ ]:
taxi_one.pick_up('Rhino')
taxi_one.passengers

#### so far the taxi class works as it supposed to do. But I have a few taxi's and really would like to instantiate them at the same time, before their shift begins so to say

In [ ]:
taxi_two = Taxi()
taxi_three= Taxi()
taxi_two.passengers

In [ ]:
taxi_three.passengers

#### Both lists are empty, so far so good let's pick up a passenger

In [ ]:
taxi_two.pick_up('George')
taxi_two.passengers

#### Now hopefully we can agree that taxi_three has no passengers

In [ ]:
taxi_three.passengers

#### However George was picked up by taxi_three too, let's get rid off George in that taxi

In [ ]:
taxi_three.drop_off('George')
taxi_two.passengers

#### By George! George has gone from both taxi's
#### As I stated this is quite the nefarious problem for Python to solve. The root cause is that Python has only two storables, primitives and pointers. 
#### What happens is that both taxi_two and taxi_three passenger list points to the same memory location, if you do an operation on that list both taxi's will have the result of that operation, for it is the same list. The class is after all defined with an empty list as a default. Taxi_one overrides this by using its own list at creation and does not have this problem.
#### Now for Python to solve this wouldn't be easy this is very deep in the core of Python. So they created a work around a solution for wanting to instantiate objects with empty lists, dataclasses' field function.
#### Let's use our FrameWork to build the same class

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Taxi:
    passengers:list[str]= field(default_factory=list)
    
    def pick_up(self,name:str):
        self.passengers.append(name)
    
    def drop_off(self,name:str):
        self.passengers.remove(name)

In [ ]:
taxi_two = Taxi()
taxi_three = Taxi()

In [ ]:
taxi_two.pick_up('George')
taxi_two.passengers

In [ ]:
taxi_three.passengers

#### The problem is solved

## **Special Methods**
#### Dataclasses allow for the easy use of Python's special methods. 
#### Special methods allow you to better leverage the Python Data Model in your programming. 
#### The use of special methods allows you to:
#### 1 make better use the rich Python standard library and the Python Data Model
#### 2 standard operations can be used, called with standard function and can have have the same name throughout classes
#### 3 create clear and concise API's that other programmers will understand. 
#### You have of course observed that all example classes have `__str__()` method. This is a special method, Python has 80 of them. 
#### You don't call these 80 special methods yourself, they are linked with functions like `len()` and `print()` or operators like `+` and `%`. 
#### Other programmers, serious Python developers, expect you to have used these special methods. Most of these special methods are part of the object class. Google expected people to make explicit that any base class would inherit from object, `class BaseClass(object):` so you would have access to most of these  special methods. 
#### However with the advent dataclasses this work method is obsolete. Dataclasses gives you access to all of them without having to inherit object explicitely. No base classes or metaclasses are used by Data Classes. Users of these classes are free to use inheritance and metaclasses without any interference from Data Classes. The decorated classes are truly “normal” Python classes. The Data Class decorator should not interfere with any usage of the class.
#### dataclasses comes with many special methods predefined for you, eg. `__repr__()` or `__data__()`. So that you can call on a clear` help()` functions for your classes or a clear `repr()`.
#### others like `__hash__()` you would have to implement yourself.

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Taxi:
    passengers:list[str]= field(default_factory=list)
    
    def pick_up(self,name:str):
        self.passengers.append(name)
        
    def drop_off(self,name:str):
        self.passengers.remove(name)
        
    def __len__(self):
        return len(self.passengers)
        


In [ ]:
taxi = Taxi()
taxi

In [ ]:
taxi.pick_up('Ente')
taxi

In [ ]:
taxi.pick_up('Croc')
taxi

In [ ]:
len(taxi)

#### For more information I will refer you to PEP 557 https://peps.python.org/pep-0557/ 

## **The Python Data Model**
#### Every programmer in Python should know the Python data model. Of course this is something you build knowledge off as you gain experience, but it doesn't hurt to read it a few times in the Python documentation. 
#### see https://docs.python.org/3/reference/datamodel.html#